In [4]:
# %load quants/4_emailtutorial.py
import os
import smtplib
import imghdr
from email.message import EmailMessage

import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import time

EMAIL_ADDRESS = os.environ.get('EMAIL_USER')
EMAIL_PASSWORD = os.environ.get('EMAIL_PASS')

msg = EmailMessage()

yf.pdr_override() # <== that's all it takes :-)
start =dt.datetime(2018,12,1)
now = dt.datetime.now()

stock="QQQ"
TargetPrice=180

msg['Subject'] = 'Alert on '+ stock+'!'
msg['From'] = EMAIL_ADDRESS
msg['To'] = 'rimoglen@terpmail.umd.edu'

alerted=False

while 1:

	df = pdr.get_data_yahoo(stock, start, now)
	currentClose=df["Adj Close"][-1]

	condition=currentClose>TargetPrice

	if(condition and alerted==False):

		alerted=True

		message=stock +" Has activated the alert price of "+ str(TargetPrice) +\
		 "\nCurrent Price: "+ str(currentClose)

		print(message)
		msg.set_content(message)

		files=[r"C:\Users\richard\Documents\KIW\Twitter\2-22\Fundamental List 2-22.xlsx"]

		for file in files:
			with open(file,'rb') as f:
				file_data=f.read()
				file_name="FundamentalList.xlsx"

				msg.add_attachment(file_data, maintype="application",
					subtype='ocetet-stream', filename=file_name)


		with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
		    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
		    smtp.send_message(msg)

		    print("completed")
	else:
		print("No new alerts")
	time.sleep(60)

In [5]:
# %load quants/4_GL_loop.py
import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr

yf.pdr_override() # <== that's all it takes :-)
start =dt.datetime(1980,12,1)
now = dt.datetime.now()
stock=""

stock = input("Enter the stock symbol : ") 
while stock != "quit":
  df = pdr.get_data_yahoo(stock, start, now)

  df.drop(df[df["Volume"]<1000].index, inplace=True)

  dfmonth=df.groupby(pd.Grouper(freq="M"))["High"].max()

  glDate=0
  lastGLV=0
  currentDate=""
  curentGLV=0
  for index, value in dfmonth.items():
    if value > curentGLV:
      curentGLV=value
      currentDate=index
      counter=0
    if value < curentGLV:
      counter=counter+1

      if counter==3 and ((index.month != now.month) or (index.year != now.year)):
          if curentGLV != lastGLV:
            print(curentGLV)
          glDate=currentDate
          lastGLV=curentGLV
          counter=0

  if lastGLV==0:
    message=stock+" has not formed a green line yet"
  else:
    message=("Last Green Line: "+str(lastGLV)+" on "+str(glDate))

  print(message)
  stock = input("Enter the stock symbol : ") 




In [6]:
# %load quants/4_GLVexcel.py
import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)
start =dt.datetime(1980,12,1)
now = dt.datetime.now()
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import Workbook
import os

root = Tk()
ftypes = [(".xlsm","*.xlsx",".xls")]
ttl  = "Title"
dir1 = 'C:\\'
#Input a file with symbols in first column
root.fileName = askopenfilename(filetypes = ftypes, initialdir = dir1, title = ttl)
print (root.fileName)

GLlistDATA = pd.DataFrame(columns=['Stock', "Last Greenline", "Date"])

a = pd.read_excel(root.fileName, index_col=0, index_row=0, header=None)
for index, value in a.iterrows():
  stock=index
  df = pdr.get_data_yahoo(stock, start, now)
  df.drop(df[df["Volume"]<1000].index, inplace=True)
  dfmonth=df.groupby(pd.Grouper(freq='M'))['High'].max()
  DOH=""
  GLV=0
  DOHc=""
  GLVc=0
  counter=0
  for index, value in dfmonth.items():
    if value> GLVc:
      GLVc=value
      DOHc=index
      counter=0
    if value < GLVc:
      counter=counter+1
      if counter==3 and ((index.month != now.month) or (index.year != now.year) ):
        GLV=GLVc
        counter=0
        DOH=DOHc
        print(str(DOH))
  GLlistDATA = GLlistDATA.append({'Stock': stock,'Greenline': GLV, 'Date': DOH}, ignore_index=True)
 
print(GLlistDATA)
newFile=os.path.dirname(root.fileName)+"/output.xlsx"
print(newFile)

writer = ExcelWriter(newFile)
GLlistDATA.to_excel(writer,'Sheet1')
writer.save() 


In [8]:
# %load quants/5_smatest.py

import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import statistics
import numpy as np
import matplotlib.ticker as mticker

yf.pdr_override() #activate yahoo finance workaround
year=1980
start =dt.datetime(year,1,1) #Sets start point of dataframe
now = dt.datetime.now() #Sets end point of dataframe

stock = input("Enter the stock symbol : ") #Asks for stock ticker

while stock != "quit": #Runs this loop until user enters 'quit' (can do many stocks in a row)

  fig, ax1 = plt.subplots() #Create Plots


  df = pdr.get_data_yahoo(stock, start, now) #Fetches stock price data, saves as data frame

  sma = int(input("Enter the sma : ")) #Asks for stock ticker
  limit= int(input("Enter Warning Limit : "))
  df['SMA'+str(sma)] = df.iloc[:,4].rolling(window=sma).mean() #calculates sma and creates a column in the dataframe
  df['PC'] = ((df["Adj Close"]/df['SMA'+str(sma)])-1)*100

  mean =df["PC"].mean()
  stdev=df["PC"].std()
  current=df["PC"][-1]
  yday=df["PC"][-2]



  print(str(current))

  print("Mean: "+str(mean))
  print("Standard Dev: "+str(stdev))

  
  bins = np.arange(-100, 100, 1) # fixed bin size

  plt.xlim([df["PC"].min()-5, df["PC"].max()+5])

  plt.hist(df["PC"], bins=bins, alpha=0.5)
  plt.title(stock+"-- % From "+str(sma)+" SMA Histogram since "+str(year))
  plt.xlabel('Percent from '+str(sma)+' SMA (bin size = 1)')
  plt.ylabel('Count')

  plt.axvline( x=mean, ymin=0, ymax=1, color='k', linestyle='--')
  plt.axvline( x=stdev+mean, ymin=0, ymax=1, color='gray', alpha=1, linestyle='--')
  plt.axvline( x=2*stdev+mean, ymin=0, ymax=1, color='gray',alpha=.75, linestyle='--')
  plt.axvline( x=3*stdev+mean, ymin=0, ymax=1, color='gray', alpha=.5, linestyle='--')
  plt.axvline( x=-stdev+mean, ymin=0, ymax=1, color='gray', alpha=1, linestyle='--')
  plt.axvline( x=-2*stdev+mean, ymin=0, ymax=1, color='gray',alpha=.75, linestyle='--')
  plt.axvline( x=-3*stdev+mean, ymin=0, ymax=1, color='gray', alpha=.5, linestyle='--')

  plt.axvline( x=current, ymin=0, ymax=1, color='r')
  plt.axvline( x=yday, ymin=0, ymax=1, color='blue')

  ax1.xaxis.set_major_locator(mticker.MaxNLocator(14)) #add more x axis labels


  fig2, ax2 = plt.subplots() #Create Plots

  df=df[-150:]

  df['PC'].plot(label='close',color='k')
  plt.title(stock+"-- % From "+str(sma)+" Over last 100 days")
  plt.xlabel('Date') 
  plt.ylabel('Percent from '+str(sma)+' EMA')
  ax2.xaxis.set_major_locator(mticker.MaxNLocator(8)) #add more x axis labels
  plt.axhline( y=limit, xmin=0, xmax=1, color='r')

  plt.show()

  
  stock = input("Enter the stock symbol : ") #Asks for new stock



In [10]:
%%writefile quants/6_RiskManage.py

import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import statistics
import time
import matplotlib.pyplot as plt


yf.pdr_override() # Activate yahoo finance workaround
now = dt.datetime.now() #Runs until todays date
start =dt.datetime(2019,1,1)

AvgGain=15 #Enter Your Average Gain %
AvgLoss=5 #Enter Your Average Loss %
smaUsed=[50,200]
emaUsed=[21]



stock = input("Enter the stock symbol (enter 'quit' to stop): ") #Query user for stock ticker


while stock != "quit": #Runs this loop until user enters 'quit' (can do many stocks in a row)
    df = pdr.get_data_yahoo(stock, start, now)
    close=df["Adj Close"][-1]
    maxStop=close*((100-AvgLoss)/100)
    Target1R=round(close*((100+AvgGain)/100),2)
    Target2R=round(close*(((100+(2*AvgGain))/100)),2)
    Target3R=round(close*(((100+(3*AvgGain))/100)),2)

    for x in smaUsed:
        sma=x
        df["SMA_"+str(sma)]=round(df.iloc[:,4].rolling(window=sma).mean(),2)
    for x in emaUsed:
        ema=x
        df['EMA_'+str(ema)] = round(df.iloc[:,4].ewm(span=ema,adjust=False).mean(),2)

    sma50=round(df["SMA_50"][-1],2)
    sma200=round(df["SMA_200"][-1],2)
    ema21=round(df["EMA_21"][-1],2)
    low5=round(min(df["Low"].tail(5)),2)

    pf50=round(((close/sma50)-1)*100,2)
    check50=df["SMA_50"][-1]>maxStop
    pf200=round(((close/sma200)-1)*100,2)
    check200=((close/df["SMA_200"][-1])-1)*100>100
    pf21=round(((close/ema21)-1)*100,2)
    check21=df["EMA_21"][-1]>maxStop 
    pfl=round(((close/low5)-1)*100,2)
    checkl=low5>maxStop 

    print()
    print("Current Stock: "+stock+" Price: "+str(round(close,2)))
    print("21 EMA: "+str(ema21)+ " | 50 SMA: "+str(sma50)+ " | 200 SMA: "+str(sma200)+ " | 5 day Low: "+str(low5))
    print("-------------------------------------------------")
    print("Max Stop: "+str(round(maxStop,2)))
    print("Price Targets:") 
    print("1R: "+str(Target1R))
    print("2R: "+str(Target2R))
    print("3R: "+str(Target3R))
    print("From 5 Day Low "+ str(pfl)+ "% -Within Max Stop: "+str(checkl))
    print("From 21 day EMA "+ str(pf21)+ "% -Within Max Stop: "+str(check21))
    print("From 50 day SMA "+ str(pf50)+ "% -Within Max Stop: "+str(check50))
    print("From 200 Day SMA "+ str(pf200)+ "% -In Danger Zone (Over 100% from 200 SMA): "+str(check200))
    print()

    stock = input("Enter the stock symbol (enter 'quit' to stop): ") #query user for next stock

Overwriting quants/6_RiskManage.py


In [ ]:
%run quants/6_RiskManage.py